In [1]:
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm

In [ ]:
with open('../data/j1939_faults_true.csv', 'w') as out_file:
    with open('../data/J1939Faults.csv') as in_file:
        for line in in_file.readlines():
            if 'True' in line:
                out_file.write(line)

In [2]:
col_names = [
    "RecordID",
    "ESS_Id",
    "EventTimeStamp",
    "eventDescription",
    "actionDescription",
    "ecuSoftwareVersion",
    "ecuSerialNumber",
    "ecuModel",
    "ecuMake",
    "ecuSource",
    "spn",
    "fmi",
    "active",
    "activeTransitionCount",
    "faultValue",
    "EquipmentID",
    "MCTNumber",
    "Latitude",
    "Longitude",
    "LocationTimeStamp"
]

In [3]:
use_cols = [
    "RecordID",
    "ESS_Id",
    "EventTimeStamp",
    "eventDescription",
    "ecuSoftwareVersion",
    "ecuSerialNumber",
    "ecuModel",
    "ecuMake",
    "ecuSource",
    "spn",
    "fmi",
    "active",
    "activeTransitionCount",
    "EquipmentID",
    "MCTNumber",
    "Latitude",
    "Longitude",
    "LocationTimeStamp"
]

In [4]:
df = pd.read_csv('../data/j1939_faults_true.csv', names = col_names, usecols = use_cols)

C:\Users\maggi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,17,True,2,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,2,True,127,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
3,6,990431,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,True,1,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59.000
4,7,990439,2015-02-21 11:40:52.000,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,17,True,2,1597,105344243,36.902916,-86.436481,2015-02-21 11:41:29.000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608454 entries, 0 to 608453
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   RecordID               608454 non-null  int64  
 1   ESS_Id                 608454 non-null  int64  
 2   EventTimeStamp         608454 non-null  object 
 3   eventDescription       574302 non-null  object 
 4   ecuSoftwareVersion     460552 non-null  object 
 5   ecuSerialNumber        437004 non-null  object 
 6   ecuModel               576200 non-null  object 
 7   ecuMake                576200 non-null  object 
 8   ecuSource              608454 non-null  int64  
 9   spn                    608454 non-null  int64  
 10  fmi                    608454 non-null  int64  
 11  active                 608454 non-null  bool   
 12  activeTransitionCount  608454 non-null  int64  
 13  EquipmentID            608454 non-null  object 
 14  MCTNumber              608454 non-nu

In [7]:
df['EquipmentID'] = df['EquipmentID'].astype(str)

In [8]:
df = df[df['EquipmentID'].str.len()<=5]

In [9]:
df = df[~((round(df['Latitude'], 3) == 36.067) & (round(df['Longitude'],3) == -86.435))]


In [10]:
df = df[~((round(df['Latitude'], 3) == 35.588) & (round(df['Longitude'],3) == -86.444))]


In [11]:
df = df[~((round(df['Latitude'], 3) == 36.195) & (round(df['Longitude'],3) == -83.175))]

In [13]:
df[['EventTimeStamp', 'LocationTimeStamp']] = df[['EventTimeStamp', 'LocationTimeStamp']].apply(pd.to_datetime)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 577231 entries, 0 to 608453
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   RecordID               577231 non-null  int64         
 1   ESS_Id                 577231 non-null  int64         
 2   EventTimeStamp         577231 non-null  datetime64[ns]
 3   eventDescription       545727 non-null  object        
 4   ecuSoftwareVersion     437490 non-null  object        
 5   ecuSerialNumber        414970 non-null  object        
 6   ecuModel               547182 non-null  object        
 7   ecuMake                547182 non-null  object        
 8   ecuSource              577231 non-null  int64         
 9   spn                    577231 non-null  int64         
 10  fmi                    577231 non-null  int64         
 11  active                 577231 non-null  bool          
 12  activeTransitionCount  577231 non-null  int6

In [16]:
df.to_csv('../data/filtered_df.csv', index = False)